In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
from copy import deepcopy

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import equinox as eqx 

In [ ]:
from rhmag.utils.final_data_evaluation import (
    FINAL_MATERIALS, TestSet, ResultSet, predict_test_scenarios, validate_result_set, visualize_result_set
)
from rhmag.utils.model_evaluation import reconstruct_model_from_file, get_exp_ids

In [ ]:
import matplotlib as mpl
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Helvetica']})
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 10})
mpl.rcParams['text.latex.preamble']=r"\usepackage{bm}\usepackage{amsmath}\usepackage{upgreek}"

---

## Gather data:

In [ ]:
FINAL_MATERIALS

In [ ]:
exp_ids_all_seeds={}
exp_name="pareto-front-f32"
for material_name in FINAL_MATERIALS:
    print("MATERIAL:", material_name)
    mat_ids=[element for element in sorted(get_exp_ids(material_name=material_name, model_type=None)) if exp_name in element]
    mat_ids_unique= list(set(mat_ids))
    [print("    " + f"'{element}'") for element in mat_ids_unique]
    #print(len(get_exp_ids(material_name=material_name, model_type=None)))
    print()
    exp_ids_all_seeds[material_name]=mat_ids_unique

#sorted(get_exp_ids(material_name="E", model_type=None))

In [ ]:
test_data = {material_name: TestSet.from_material_name(material_name) for material_name in FINAL_MATERIALS}

In [ ]:
from rhmag.utils.pretest_evaluation import create_multilevel_df
from rhmag.utils.final_data_evaluation import evaluate_test_scenarios

In [ ]:
# metrics_per_material = {}
all_results = []
for material_name, exp_ids in exp_ids_all_seeds.items():
    test_set = test_data[material_name]
    for exp_id in exp_ids:
        model = reconstruct_model_from_file(exp_id)
        model_params = model.n_params
        seed = exp_id.split("seed")[-1]
        model_type = exp_id.split("_")[1]
        exp_name = exp_id.split("_")[2]
        num_id = exp_id.split("_")[-2]
        metrics_per_sequence = evaluate_test_scenarios(model, test_set)
        metrics = ['sre_avg', 'sre_95th', 'nere_avg', 'nere_95th']
        averages = {m: sum(d[m] for d in metrics_per_sequence.values()) / len(metrics_per_sequence) for m in metrics}
        all_results.append(
                {
                    "exp_id_without_seed": exp_id.rpartition('_')[0],
                    "exp_id": exp_id,
                    "exp_name": exp_name,
                    "num_id": num_id,
                    "material": material_name,
                    "model_type": model_type,
                    "seed": seed,
                    "n_params": model_params,
                    "sre_avg": averages["sre_avg"],
                    "sre_95th": averages["sre_95th"],
                    "nere_avg": averages["nere_avg"],
                    "nere_95th": averages["nere_95th"],
                }
            )
df_results = pd.DataFrame(all_results)

In [ ]:
df_results

In [ ]:
# df_best_seeds = df_results.loc[df_results.groupby('exp_id_without_seed')['sre_95th'].idxmin()]

# df_best_seeds = df_best_seeds.reset_index(drop=True)
# df_model_comparison_best = df_best_seeds.groupby("model_type").mean(numeric_only=True).reset_index()

# df_results_spec=df_results[(df_results["n_params"] < 500) & (df_results["material"] != "A")]
# df_model_comparison_spec = df_results_spec.groupby("model_type").mean(numeric_only=True).reset_index()

In [ ]:
df_model_comparison = df_results.groupby("model_type").mean(numeric_only=True).reset_index()

### Other Teams

In [ ]:
EXTERNAL_TEAMS = [
    {
        "model_type": "Fuzhou", 
        "n_params": 63022, 
        "sre_95th": 12.42 / 100, 
        "nere_95th": 1.91 / 100
    },
    {
        "model_type": "Hefei", 
        "n_params": 6564, 
        "sre_95th": 9.17 / 100, 
        "nere_95th": 1.57 / 100
    },
    {
        "model_type": "Cambridge", 
        "n_params": 1182, 
        "sre_95th": 37.01 / 100, 
        "nere_95th": 9.24 / 100
    },
    {
        "model_type": "Sydney", 
        "n_params": 4777, 
        "sre_95th": 24.63 / 100, 
        "nere_95th": 5.44 / 100
    },
    {
        "model_type": "Gatech", 
        "n_params": 83, 
        "sre_95th": 38.76 / 100, 
        "nere_95th": 7.97 / 100
    },
    {
        "model_type": "SiegenPaderborn", 
        "n_params": 325, 
        "sre_95th": 20.43 / 100, 
        "nere_95th": 3.41 / 100
    },
    ]


In [ ]:
df_external = pd.DataFrame(EXTERNAL_TEAMS)

In [ ]:
df_combined = pd.concat([df_model_comparison, df_external], ignore_index=True)

In [ ]:
df_combined

## Plot:

In [ ]:
import seaborn as sns

In [ ]:
def visualize_pareto_final(df, metrics, color_own="blue", color_others="gray", scale_log_metric=True, scale_log_size=True, highlighted_type="GRU", sharex="col", sharey="row"):
    #sns.set_theme(style="whitegrid")
    
    df = df.copy()
    df["IsOwn"] = df["model_type"].apply(lambda x: highlighted_type in str(x))
    
    fig, axs = plt.subplots(nrows=1, ncols=len(metrics), sharex=sharex, sharey=sharey, figsize=(7.167, 7.167 / 2), squeeze=False)
    
    for i, metric in enumerate(metrics):
        ax = axs[0, i]
        target_col = f"{metric}_95th"
        
        for is_own, group_df in df.groupby("IsOwn"):
            current_color = color_own if is_own else color_others
            
            sns.scatterplot(
                data=group_df,
                x=target_col, 
                y="n_params",
                color=current_color,
                marker="o" if is_own else "s", 
                s=20,
                alpha=0.8,
                ax=ax,
                zorder=3
            )
            
            if not is_own:
                for _, row in group_df.iterrows():
                    ax.text(
                        row[target_col] * 1.05, 
                        row["n_params"], 
                        str(row["model_type"]),
                        fontsize=10,
                        alpha=1,
                        va='center', 
                        color=current_color,
                        fontweight='normal'
                    )

        if scale_log_metric:
            ax.set_xscale("log")
        if scale_log_size:
            ax.set_yscale("log")
            
        unique_params = sorted(df["n_params"].unique().astype(int))
        ax.set_yticks(unique_params)
        ax.yaxis.set_major_formatter(plt.ScalarFormatter())
        ax.xaxis.set_major_formatter(plt.ScalarFormatter())
        
        ax.set_yticklabels(unique_params, rotation=0, fontsize=9) #ha='right')
        ax.tick_params(which='major', axis="y", direction='in')
        ax.tick_params(which='both', axis="x", direction='in')
        ax.yaxis.minorticks_off()
        ax.xaxis.minorticks_off()

        if i == 0:
            ax.set_xticks([0.2, 0.3, 0.4, 0.5, 0.6], minor=True)
            ax.set_xticks([0.1], minor=False)
    
            ax.xaxis.set_minor_formatter(plt.ScalarFormatter())

            ax.set_xlim(0.08, 0.75)

        if i == 1:
            ax.set_xticks([0.02, 0.05, 0.2], minor=True)
            ax.set_xticks([0.1], minor=False)
    
            ax.xaxis.set_minor_formatter(plt.ScalarFormatter())

            ax.set_xlim(0.013, 0.22)
        
        if i == 0:
            ax.set_ylabel("\# Model params.", fontsize=9)
        
        label_map = {"sre": "95-th percentile SRE", "nere": "95-th percentile NERE"}
        ax.set_xlabel(label_map.get(metric, metric), fontsize=9)
        
        ax.grid(True, which="both", ls="--", alpha=0.3)

    plt.tight_layout()
    return fig, axs

In [ ]:
fig, axs=visualize_pareto_final(
    df_combined,
    ["sre","nere"],
    color_own="orange",
    color_others="black",
    scale_log_metric=True,
    scale_log_size=True,
    highlighted_type="GRU",
    sharex="col",
    sharey="row"
)
# plt.savefig("pareto_comparison.pdf", bbox_inches="tight")
# plt.savefig("pareto_comparison.png", bbox_inches="tight", dpi=400)